### Read data

In [1]:
import os
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import manifold
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from collections import defaultdict

In [2]:
os.getcwd()

'C:\\Users\\as743ce\\Exercise'

In [3]:
data = pd.read_excel('C:\\Users\\as743ce\\Exercise\\Dummy.xlsx', header=0)

In [4]:
header = list(data.columns.values)

In [ ]:
print(header)

### Create console

In [11]:
console = {}
console_meta = {}
list_p = list()
list_d = list()
list_n = list()
list_q = list()
for col in header:
    console[col] = data[col].unique().tolist()
    n = data[col].count()
    q = len(data[col].unique().tolist())
    p = q / n
    list_p.append(p)
    list_n.append(n)
    list_q.append(q)
    if is_numeric_dtype(data[col]):
        list_d.append('Numeric')
    else:
        list_d.append('Categoric')
        
console_meta['Count'] = list_n
console_meta['Unique'] = list_q
console_meta['Div'] = list_p
console_meta['Type'] = list_d

In [12]:
print(console)
print(console_meta)

{'ID': [1000000, 1000001, 1000002, 1000003, 1000004, 1000005, 1000006, 1000007, 1000008, 1000009, 1000010, 1000011, 1000012, 1000013, 1000014, 1000015, 1000016, 1000017, 1000018, 1000019, 1000020, 1000021, 1000022, 1000023, 1000024, 1000025, 1000026, 1000027, 1000028, 1000029, 1000030, 1000031, 1000032, 1000033, 1000034, 1000035, 1000036, 1000037, 1000038, 1000039, 1000040, 1000041, 1000042, 1000043, 1000044, 1000045, 1000046, 1000047, 1000048, 1000049, 1000050, 1000051, 1000052, 1000053, 1000054, 1000055, 1000056, 1000057, 1000058, 1000059, 1000060, 1000061, 1000062, 1000063, 1000064, 1000065, 1000066, 1000067, 1000068, 1000069, 1000070, 1000071, 1000072, 1000073, 1000074, 1000075, 1000076, 1000077, 1000078, 1000079, 1000080, 1000081, 1000082, 1000083, 1000084, 1000085, 1000086, 1000087, 1000088, 1000089, 1000090, 1000091, 1000092, 1000093, 1000094, 1000095, 1000096, 1000097, 1000098, 1000099], 'Seller': ['Zumbeispiel GmbH', '3rd party', 'bspw bank AG'], 'Sellercode': ['ZBG', '3RD', '

### Normalizing data

In [ ]:
data_norm = data.copy()

In [ ]:
data_norm.columns = np.arange(len(data_norm.columns))

In [ ]:
data_norm.head()

In [ ]:
d = defaultdict(list)
l = 1
r = list()
for col in data_norm.columns:
    for restcol in data_norm.columns:
        if restcol > col:
            for idx in data_norm.index:
                if data_norm.loc[idx, restcol] not in d[data_norm.loc[idx, col]]:
                    d[data_norm.loc[idx, col]].append(data_norm.loc[idx, restcol])
            for i in d:
                if len(d[i]) > l:
                    l = len(d[i])
            if l == 1:
                r.append(restcol)
            d = defaultdict(list)
data_norm.drop(columns=r, inplace=True)
data_norm.columns = np.arange(len(data_norm.columns))

In [ ]:
data_norm.head()

### Outlier scores

#### Clusters

In [ ]:
hindex = list(data_norm.columns.values)

In [ ]:
data_outlier = data_norm.reset_index(drop=True)

In [ ]:
data_outlier.sort_values(hindex, inplace=True)

In [ ]:
data_outlier.reset_index(drop=True, inplace=True)

In [ ]:
data_outlier['Cluster'] = np.nan

In [ ]:
cluster = 1
for index,row in data_outlier.iterrows():
    if index != 0:
        vari = 0
        for hdx in hindex:
            if data_outlier.loc[index, hdx] != data_outlier.loc[index-1, hdx]:
                vari += 1
        if vari != 0:
            cluster += 1
        data_outlier.loc[index, 'Cluster'] = cluster
    else:
        data_outlier.loc[index, 'Cluster'] = cluster

#### Unique items

In [ ]:
data_unique = data_outlier.copy()

In [ ]:
data_unique['Count'] = np.nan

In [ ]:
count = 1
for index,row in data_unique.iterrows():
    if index < data_unique.index[-1]:
        if data_unique.loc[index, 'Cluster'] == data_unique.loc[index+1, 'Cluster']:
            data_unique.drop(index, axis=0, inplace = True)
            count += 1
        else:
            data_unique.loc[index, 'Count'] = count
            count = 1           
    else:
        data_unique.loc[index, 'Count'] = count
        count = 1

In [ ]:
data_unique.sort_values('Count', inplace=True)

In [ ]:
data_unique.reset_index(drop=True, inplace=True)

#### ReCluster

In [ ]:
data_unique['ReCluster'] = np.nan

In [ ]:
for idx in data_unique.index:
    for idxr in reversed(data_unique.index):
        if idxr > idx:
            vari = 0
            for hdx in hindex:
                if data_unique.loc[idx, hdx] != data_unique.loc[idxr, hdx]:
                    vari += 1
            if vari == 1:
                data_unique.loc[idx, 'ReCluster'] = data_unique.loc[idxr, 'Cluster']
    if pd.isnull(data_unique.loc[idx, 'ReCluster']):
        data_unique.loc[idx, 'ReCluster'] = data_unique.loc[idx, 'Cluster']

In [ ]:
data_unique.head(18)

#### Outlier scores

In [ ]:
data_unique['OutlierScore'] = np.nan

In [ ]:
for idx in data_unique.index:
    ReCluster = data_unique.loc[idx, 'ReCluster']
    CountTotal = data_unique[data_unique['ReCluster'] == ReCluster]['Count'].sum()
    CountSub = data_unique.loc[idx, 'Count']
    data_unique.loc[idx, 'OutlierScore'] = 1 - (CountSub / CountTotal)

In [ ]:
data_unique.head()

### Reduce data in scope

In [ ]:
dimensions = ['Seller', 'Buyer', 'Description']

In [ ]:
data_red = pd.DataFrame(index = data.index).copy()
for d in dimensions:
    data_red[d] = data[d]

In [ ]:
data_red.head()

In [ ]:
data_list = pd.DataFrame(index = data.index)
data_list['list'] = data_red.values.tolist()

In [ ]:
data_list.head()

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_list['list'].astype(str))

In [ ]:
data_onehot = pd.DataFrame(index = data.index, columns = vectorizer.get_feature_names(), data = X.toarray())

In [ ]:
data_onehot.head()

In [ ]:
mds3 = manifold.MDS(n_components=3, n_init=1, max_iter=50)
data_mds3 = mds3.fit_transform(data_onehot)

In [ ]:
fig = plt.figure()
axe = Axes3D(fig)

axe.scatter(data_mds3[:,0], data_mds3[:,1], data_mds3[:,2], cmap='BrBG')
plt.show()